In [111]:
import pandas as pd
import numpy as np
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [55]:
movies = pd.read_csv("dataset/tmdb_5000_movies.csv")
credits = pd.read_csv("dataset/tmdb_5000_credits.csv")

In [56]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [57]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [58]:
movies = movies.merge(credits, on='title')

In [59]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [60]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

In [61]:
# genres
# id
# keywords
# original_language
# title
# overview
# runtime
# cast
# crew

In [62]:
movies = movies[['id', 'title', 'overview', 'genres', 'keywords', 'original_language', 'cast', 'crew']]

In [63]:
movies.head(1)

,id,title,overview,genres,keywords,original_language,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [64]:
movies.dropna(inplace=True)

In [65]:
movies.isnull().sum()

id                   0
title                0
overview             0
genres               0
keywords             0
original_language    0
cast                 0
crew                 0
dtype: int64

In [66]:
movies.duplicated().sum()

0

In [67]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [68]:
def get_geners_or_keywords(obj):
    my_list = []
    for i in ast.literal_eval(obj):
        my_list.append(i['name'])
    return my_list

In [69]:
movies['genres'] = movies['genres'].apply(get_geners_or_keywords)

In [70]:
movies.head(1)

,id,title,overview,genres,keywords,original_language,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [71]:
movies['keywords'] = movies['keywords'].apply(get_geners_or_keywords)

In [72]:
movies.head(1)

,id,title,overview,genres,keywords,original_language,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [73]:
def get_cast(obj):
    my_list = []
    count = 0
    for i in ast.literal_eval(obj):
        if(count != 3):
            my_list.append(i['name'])
        else:
            break
    return my_list

In [74]:
movies['cast'] = movies['cast'].apply(get_cast)

In [75]:
movies.head(1)

,id,title,overview,genres,keywords,original_language,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,"[Sam Worthington, Zoe Saldana, Sigourney Weave...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [76]:
def get_director(obj):
    my_list = []
    for i in ast.literal_eval(obj):
        if(i['department'] == "Directing"):
            my_list.append(i['name'])
    return my_list

In [77]:
movies['crew'] = movies['crew'].apply(get_director)

In [78]:
movies.head(1)

,id,title,overview,genres,keywords,original_language,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,"[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron]


In [79]:
movies['overview'] = movies['overview'].apply(lambda x: x.split())
movies['original_language'] = movies['original_language'].apply(lambda x: x.split())

In [80]:
movies.head(1)

,id,title,overview,genres,keywords,original_language,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",[en],"[Sam Worthington, Zoe Saldana, Sigourney Weave...",[James Cameron]


In [81]:
movies['genres'] = movies['genres'].apply(lambda x: [i.replace(" ", "-") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x: [i.replace(" ", "-") for i in x])
movies['cast'] = movies['cast'].apply(lambda x: [i.replace(" ", "-") for i in x])
movies['crew'] = movies['crew'].apply(lambda x: [i.replace(" ", "-") for i in x])

In [82]:
movies.head(1)

,id,title,overview,genres,keywords,original_language,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science-Fiction]","[culture-clash, future, space-war, space-colon...",[en],"[Sam-Worthington, Zoe-Saldana, Sigourney-Weave...",[James-Cameron]


In [83]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew'] + movies['original_language']

In [84]:
movies.head(1)

,id,title,overview,genres,keywords,original_language,cast,crew,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science-Fiction]","[culture-clash, future, space-war, space-colon...",[en],"[Sam-Worthington, Zoe-Saldana, Sigourney-Weave...",[James-Cameron],"[In, the, 22nd, century,, a, paraplegic, Marin..."


In [85]:
new_df = movies[['id', 'title', 'tags']]

In [86]:
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))

C:\Users\Admin\AppData\Local\Temp\ipykernel_12436\1824047427.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))


In [87]:
new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())

C:\Users\Admin\AppData\Local\Temp\ipykernel_12436\1380776331.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].apply(lambda x: x.lower())


In [88]:
new_df

,id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."
...,...,...,...
4804,9367,El Mariachi,el mariachi just wants to play his guitar and ...
4805,72766,Newlyweds,a newlywed couple's honeymoon is upended by th...
4806,231617,"Signed, Sealed, Delivered","""signed, sealed, delivered"" introduces a dedic..."
4807,126186,Shanghai Calling,when ambitious new york attorney sam is sent t...


In [89]:
cv = CountVectorizer(max_features=5000, stop_words='english')

In [90]:
vectors = cv.fit_transform(new_df['tags']).toarray()

In [91]:
vectors.shape

(4806, 5000)

In [92]:
cv.get_feature_names_out()

array(['000', '10', '11', ..., 'zooey', 'zoë', 'zucker'], dtype=object)

In [93]:
similarity = cosine_similarity(vectors)

In [94]:
similarity[0]

array([1.        , 0.10309929, 0.07366401, ..., 0.0378811 , 0.04235235,
       0.03092979])

In [109]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [110]:
recommend("Batman Begins")

The Dark Knight Rises
The Dark Knight
Superman
The Departed
The Fifth Element


In [112]:
pickle.dump(new_df, open("movies.pkl", 'wb'))

In [113]:
pickle.dump(similarity, open("similarity.pkl", 'wb'))